## Question 1

@Sparsha-Sherke ➜ /workspaces/llm_zoomcamp (main) $ curl localhost:9200
{
  "name" : "014078d4a505",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "ItBkWYAEQ4el2Zm5Kvgr3w",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}

## Question 2

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
from elasticsearch import Elasticsearch

In [3]:
es_client = Elasticsearch('http://localhost:9200') 

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-homework-sparsha"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-homework-sparsha'})

In [7]:
from tqdm.auto import tqdm

In [8]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████| 948/948 [00:37<00:00, 25.28it/s]


## Question 3

In [9]:
query = 'How do I execute a command in a running docker container?'

In [10]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append({
            "document": hit['_source'],
            "score": hit['_score']
        })
    
    return result_docs

def find_top_score(query):
    results = elastic_search(query)
    if results:
        top_score = results[0]["score"]
        return top_score
    else:
        return None

# Example query
query = "How do I execute a command in a running docker container?"
top_score = find_top_score(query)

print(f"Top score for the query: {top_score}")

Top score for the query: 75.54128


Output of above cell : "Top score for the query: 75.54128". Hence, the closest option 74.05 was chosen.

## Question 4

In [11]:
def search(query, course):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append({
            "question": hit['_source']['question'],
            "score": hit['_score']
        })
    
    if len(result_docs) >= 3:
        return result_docs[2]["question"]
    else:
        return None

query = "How do I execute a command in a running docker container?"
course = "machine-learning-zoomcamp"
third_question = search(query, course)

print(f"The 3rd question returned by the search engine: {third_question}")

The 3rd question returned by the search engine: How do I copy files from a different folder into docker container’s working directory?


## Question 5

In [12]:
def search(query, course):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append({
            "question": hit['_source']['question'],
            "text": hit['_source']['text']
        })
    
    context = ""
    for doc in result_docs:
        context += context_template.format(question=doc["question"], text=doc["text"]) + "\n\n"
    
    return context

query = "How do I execute a command in a running docker container?"
course = "machine-learning-zoomcamp"
context_template = """
Q: {question}
A: {text}
""".strip()

In [13]:
context = search(query, course)

print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [14]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

question = "How do I execute a command in a running docker container?"
context = search(query, course)

prompt = prompt_template.format(question=question, context=context)

print(prompt)

print("The length of the resulting prompt is:", len(prompt))

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

## Question 6

In [15]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [17]:
import tiktoken
encoding = tiktoken.encoding_for_model("phi3")

KeyError: 'Could not automatically map phi3 to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.'

In [18]:
import tiktoken

prompt = "How do I execute a command in a running docker container?"
encoding = tiktoken.get_encoding("cl100k_base")

print(encoding.encode(prompt))

[4438, 656, 358, 9203, 264, 3290, 304, 264, 4401, 27686, 5593, 30]


In [19]:
print(len(encoding.encode(prompt)))

12
